# Plant_seedling_classification

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import glob
from tensorflow.keras import datasets, layers, models
from keras.layers import Dropout, Input, Dense, Activation,GlobalMaxPooling2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model, load_model
from keras.optimizers import Adam
from PIL import Image, ImageOps
from keras.utils import np_utils
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## Loading Data

In [2]:
#트레인 데이터 불러오기
n = glob.glob('C:/Users/tnqls/Desktop/plant/train/*/*.png')
ori_label = []
ori_imgs = []
for a in n:
    if a[-3:] != 'png':
        continue
    ori_label.append(a.split('\\')[-2]) #파일 경로에서 파일 제목을 식물이름으로 저장하는 방법
    open_img = Image.open(a)
    ori_imgs.append(ImageOps.fit(open_img, (48,48), Image.ANTIALIAS).convert('RGB'))

In [3]:
#데이터 확인하기
print(len(ori_imgs))
print(len(ori_label))
print(ori_imgs[0])
print(ori_label[0])

4750
4750
<PIL.Image.Image image mode=RGB size=48x48 at 0x1CE84D58D48>
Black-grass


In [4]:
#이미지 파일을 array형태로 변환
imgs = np.array([np.array(im) for im in ori_imgs])
imgs = imgs.reshape(imgs.shape[0], 48, 48, 3) / 255
lb = LabelBinarizer().fit(ori_label)
label = lb.transform(ori_label) 

In [5]:
#train데이터랑 valid데이터로 나누기
from sklearn.model_selection import train_test_split
train_img,valid_img,train_label,valid_label = train_test_split(imgs,label,test_size=0.2,random_state=0)

## 모델만들기

In [6]:
#모델 만들기(기본)
Zn_input = Input((48, 48, 3))

Zn = Conv2D(16, (3, 3))(Zn_input)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(16, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = MaxPooling2D((2, 2), strides=(2, 2))(Zn)

Zn = Conv2D(32, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(32, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)

Zn = Conv2D(64, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(64, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = MaxPooling2D((2, 2), strides=(2, 2))(Zn)

Zn = Conv2D(128, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(128, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = GlobalMaxPooling2D()(Zn)

Zn = Dense(64, activation='relu')(Zn)
Zn = Dropout(0.5)(Zn)
Zn = Dense(32, activation='relu')(Zn)
Zn = Dropout(0.5)(Zn)
Zn = Dense(12, activation='softmax')(Zn)
model_basic = Model(inputs=Zn_input, outputs=Zn)
model_basic.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4), metrics=['acc'])

In [7]:
#만들어진 모델 형태 확인하기
model_basic.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 16)        448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 16)        2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 44, 44, 16)        64        
_________________________________________________________________
activation_2 (Activation)    (None, 44, 44, 16)        0   

In [8]:
#모델 학습하기
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
earlystop = EarlyStopping(patience=10)
modelsave = ModelCheckpoint(filepath='model_basic.h5', save_best_only=True, verbose=1)
model_basic.fit(train_img, train_label, batch_size=64,epochs=150,validation_data=(valid_img, valid_label),callbacks=[annealer, earlystop, modelsave])

Train on 3800 samples, validate on 950 samples
Epoch 1/150
3800/3800 [==============================] - 31s 8ms/step - loss: 2.6074 - acc: 0.1053 - val_loss: 2.4491 - val_acc: 0.1368

Epoch 00001: val_loss improved from inf to 2.44906, saving model to model_basic.h5
Epoch 2/150
3800/3800 [==============================] - 27s 7ms/step - loss: 2.3508 - acc: 0.1642 - val_loss: 2.4633 - val_acc: 0.1368

Epoch 00002: val_loss did not improve from 2.44906
Epoch 3/150
3800/3800 [==============================] - 27s 7ms/step - loss: 2.1549 - acc: 0.2563 - val_loss: 2.4768 - val_acc: 0.1526

Epoch 00003: val_loss did not improve from 2.44906
Epoch 4/150
3800/3800 [==============================] - 27s 7ms/step - loss: 1.9929 - acc: 0.3279 - val_loss: 2.5773 - val_acc: 0.2105

Epoch 00004: val_loss did not improve from 2.44906
Epoch 5/150
3800/3800 [==============================] - 28s 7ms/step - loss: 1.8260 - acc: 0.3847 - val_loss: 3.1758 - val_acc: 0.1611

Epoch 00005: val_loss did not im

Epoch 41/150
3800/3800 [==============================] - 28s 7ms/step - loss: 0.5312 - acc: 0.8121 - val_loss: 0.4648 - val_acc: 0.8663

Epoch 00041: val_loss did not improve from 0.46021
Epoch 42/150
3800/3800 [==============================] - 29s 8ms/step - loss: 0.5226 - acc: 0.8216 - val_loss: 0.4572 - val_acc: 0.8695

Epoch 00042: val_loss improved from 0.46021 to 0.45724, saving model to model_basic.h5
Epoch 43/150
3800/3800 [==============================] - 28s 7ms/step - loss: 0.5353 - acc: 0.8095 - val_loss: 0.4645 - val_acc: 0.8589

Epoch 00043: val_loss did not improve from 0.45724
Epoch 44/150
3800/3800 [==============================] - 28s 7ms/step - loss: 0.5159 - acc: 0.8037 - val_loss: 0.4597 - val_acc: 0.8674

Epoch 00044: val_loss did not improve from 0.45724
Epoch 45/150
3800/3800 [==============================] - 29s 8ms/step - loss: 0.5274 - acc: 0.8058 - val_loss: 0.4636 - val_acc: 0.8684

Epoch 00045: val_loss did not improve from 0.45724
Epoch 46/150
3800/3

## 모델만들기 Layer(128->256)

In [9]:
#모델만들기(layer 128->256)
Zn_input = Input((48, 48, 3))
Zn = Conv2D(16, (3, 3))(Zn_input)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(16, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)

Zn = Conv2D(32, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(32, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = MaxPooling2D((2, 2), strides=(2, 2))(Zn)

Zn = Conv2D(64, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(64, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)

Zn = Conv2D(128, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(128, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = MaxPooling2D((2, 2), strides=(2, 2))(Zn)

Zn = Conv2D(256, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(256, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = GlobalMaxPooling2D()(Zn)

Zn = Dense(64, activation='relu')(Zn)
Zn = Dropout(0.5)(Zn)
Zn = Dense(32, activation='relu')(Zn)
Zn = Dropout(0.5)(Zn)
Zn = Dense(12, activation='softmax')(Zn)
model_layer = Model(inputs=Zn_input, outputs=Zn)
model_layer.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4), metrics=['acc'])
model_layer.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 46, 46, 16)        448       
_________________________________________________________________
batch_normalization_9 (Batch (None, 46, 46, 16)        64        
_________________________________________________________________
activation_9 (Activation)    (None, 46, 46, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 44, 44, 16)        2320      
_________________________________________________________________
batch_normalization_10 (Batc (None, 44, 44, 16)        64        
_________________________________________________________________
activation_10 (Activation)   (None, 44, 44, 16)        0   

In [10]:
#모델(Layer 128->256) 학습하기
#모델 학습하기
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
earlystop = EarlyStopping(patience=10)
modelsave = ModelCheckpoint(filepath='model_layer.h5', save_best_only=True, verbose=1)
model_layer.fit(train_img, train_label, batch_size=64,epochs=150,validation_data=(valid_img, valid_label),callbacks=[annealer, earlystop, modelsave])

Train on 3800 samples, validate on 950 samples
Epoch 1/150
3800/3800 [==============================] - 71s 19ms/step - loss: 2.5352 - acc: 0.1284 - val_loss: 2.4376 - val_acc: 0.1368

Epoch 00001: val_loss improved from inf to 2.43762, saving model to model_layer.h5
Epoch 2/150
3800/3800 [==============================] - 69s 18ms/step - loss: 2.2475 - acc: 0.2187 - val_loss: 2.5062 - val_acc: 0.1368

Epoch 00002: val_loss did not improve from 2.43762
Epoch 3/150
3800/3800 [==============================] - 73s 19ms/step - loss: 2.0251 - acc: 0.3037 - val_loss: 3.9145 - val_acc: 0.1368

Epoch 00003: val_loss did not improve from 2.43762
Epoch 4/150
3800/3800 [==============================] - 78s 21ms/step - loss: 1.8310 - acc: 0.3800 - val_loss: 6.4095 - val_acc: 0.1368

Epoch 00004: val_loss did not improve from 2.43762
Epoch 5/150
3800/3800 [==============================] - 71s 19ms/step - loss: 1.6783 - acc: 0.4495 - val_loss: 8.5442 - val_acc: 0.1379

Epoch 00005: val_loss did n

Epoch 41/150
3800/3800 [==============================] - 83s 22ms/step - loss: 0.3826 - acc: 0.8584 - val_loss: 0.3738 - val_acc: 0.8853

Epoch 00041: val_loss improved from 0.37521 to 0.37381, saving model to model_layer.h5
Epoch 42/150
3800/3800 [==============================] - 73s 19ms/step - loss: 0.4017 - acc: 0.8521 - val_loss: 0.3716 - val_acc: 0.8874

Epoch 00042: val_loss improved from 0.37381 to 0.37163, saving model to model_layer.h5
Epoch 43/150
3800/3800 [==============================] - 67s 18ms/step - loss: 0.3885 - acc: 0.8547 - val_loss: 0.3747 - val_acc: 0.8895

Epoch 00043: val_loss did not improve from 0.37163
Epoch 44/150
3800/3800 [==============================] - 66s 17ms/step - loss: 0.3751 - acc: 0.8632 - val_loss: 0.3736 - val_acc: 0.8863

Epoch 00044: val_loss did not improve from 0.37163
Epoch 45/150
3800/3800 [==============================] - 67s 18ms/step - loss: 0.3848 - acc: 0.8547 - val_loss: 0.3663 - val_acc: 0.8842

Epoch 00045: val_loss improve

## 모델만들기(Batch size 64->32)

In [11]:
#모델만들기(Batchsize 64->32)
model_batch = Model(inputs=Zn_input, outputs=Zn)
model_batch.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4), metrics=['acc'])
model_batch.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 46, 46, 16)        448       
_________________________________________________________________
batch_normalization_9 (Batch (None, 46, 46, 16)        64        
_________________________________________________________________
activation_9 (Activation)    (None, 46, 46, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 44, 44, 16)        2320      
_________________________________________________________________
batch_normalization_10 (Batc (None, 44, 44, 16)        64        
_________________________________________________________________
activation_10 (Activation)   (None, 44, 44, 16)        0   

In [12]:
#모델(Batchsize 64->32) 학습하기
#모델 학습하기
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
earlystop = EarlyStopping(patience=10)
modelsave = ModelCheckpoint(filepath='model_batch.h5', save_best_only=True, verbose=1)
model_batch.fit(train_img, train_label, batch_size=32,epochs=150,validation_data=(valid_img, valid_label),callbacks=[annealer, earlystop, modelsave])

Train on 3800 samples, validate on 950 samples
Epoch 1/150
3800/3800 [==============================] - 67s 18ms/step - loss: 1.2025 - acc: 0.6471 - val_loss: 2.8850 - val_acc: 0.4505

Epoch 00001: val_loss improved from inf to 2.88500, saving model to model_batch.h5
Epoch 2/150
3800/3800 [==============================] - 65s 17ms/step - loss: 1.0060 - acc: 0.6882 - val_loss: 2.1904 - val_acc: 0.5368

Epoch 00002: val_loss improved from 2.88500 to 2.19036, saving model to model_batch.h5
Epoch 3/150
3800/3800 [==============================] - 65s 17ms/step - loss: 0.9217 - acc: 0.7282 - val_loss: 1.0785 - val_acc: 0.7021

Epoch 00003: val_loss improved from 2.19036 to 1.07853, saving model to model_batch.h5
Epoch 4/150
3800/3800 [==============================] - 66s 17ms/step - loss: 0.8794 - acc: 0.7421 - val_loss: 1.1287 - val_acc: 0.6758

Epoch 00004: val_loss did not improve from 1.07853
Epoch 5/150
3800/3800 [==============================] - 66s 17ms/step - loss: 0.7864 - acc: 

## 모델만들기(이미지 사이즈 48->64)

In [13]:
#트레인 데이터 불러오기
n = glob.glob('C:/Users/tnqls/Desktop/plant/train/*/*.png')
ori_label = []
ori_imgs = []
for a in n:
    if a[-3:] != 'png':
        continue
    ori_label.append(a.split('\\')[-2]) #파일 경로에서 파일 제목을 식물이름으로 저장하는 방법
    open_img = Image.open(a)
    ori_imgs.append(ImageOps.fit(open_img, (64,64), Image.ANTIALIAS).convert('RGB'))

#이미지 파일을 array형태로 변환
imgs = np.array([np.array(im) for im in ori_imgs])
imgs = imgs.reshape(imgs.shape[0], 64, 64, 3) / 255
lb = LabelBinarizer().fit(ori_label)
label = lb.transform(ori_label)

#train데이터랑 valid데이터로 나누기
from sklearn.model_selection import train_test_split
train_img,valid_img,train_label,valid_label = train_test_split(imgs,label,test_size=0.2,random_state=0)

#모델만들기(이미지 사이즈 48->64)
Zn_input = Input((64, 64, 3))
Zn = Conv2D(16, (3, 3))(Zn_input)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(16, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = MaxPooling2D((2, 2), strides=(2, 2))(Zn)

Zn = Conv2D(32, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(32, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)

Zn = Conv2D(64, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(64, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = MaxPooling2D((2, 2), strides=(2, 2))(Zn)

Zn = Conv2D(128, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = Conv2D(128, (3, 3))(Zn)
Zn = BatchNormalization(axis = 3)(Zn)
Zn = Activation('relu')(Zn)
Zn = GlobalMaxPooling2D()(Zn)

Zn = Dense(64, activation='relu')(Zn)
Zn = Dropout(0.5)(Zn)
Zn = Dense(32, activation='relu')(Zn)
Zn = Dropout(0.5)(Zn)
Zn = Dense(12, activation='softmax')(Zn)
model_img = Model(inputs=Zn_input, outputs=Zn)
model_img.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4), metrics=['acc'])
model_img.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 62, 62, 16)        448       
_________________________________________________________________
batch_normalization_19 (Batc (None, 62, 62, 16)        64        
_________________________________________________________________
activation_19 (Activation)   (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 60, 60, 16)        2320      
_________________________________________________________________
batch_normalization_20 (Batc (None, 60, 60, 16)        64        
_________________________________________________________________
activation_20 (Activation)   (None, 60, 60, 16)        0   

In [14]:
#모델(이미지 크기) 학습하기
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
earlystop = EarlyStopping(patience=10)
modelsave = ModelCheckpoint(filepath='model_img.h5', save_best_only=True, verbose=1)
model_img.fit(train_img, train_label, batch_size=64,epochs=150,validation_data=(valid_img, valid_label),callbacks=[annealer, earlystop, modelsave])

Train on 3800 samples, validate on 950 samples
Epoch 1/150
3800/3800 [==============================] - 55s 15ms/step - loss: 2.6361 - acc: 0.1063 - val_loss: 2.4718 - val_acc: 0.0947

Epoch 00001: val_loss improved from inf to 2.47184, saving model to model_img.h5
Epoch 2/150
3800/3800 [==============================] - 54s 14ms/step - loss: 2.3358 - acc: 0.1742 - val_loss: 2.5185 - val_acc: 0.1095

Epoch 00002: val_loss did not improve from 2.47184
Epoch 3/150
3800/3800 [==============================] - 54s 14ms/step - loss: 2.1415 - acc: 0.2716 - val_loss: 2.7843 - val_acc: 0.1337

Epoch 00003: val_loss did not improve from 2.47184
Epoch 4/150
3800/3800 [==============================] - 54s 14ms/step - loss: 1.9672 - acc: 0.3297 - val_loss: 2.5409 - val_acc: 0.1337

Epoch 00004: val_loss did not improve from 2.47184
Epoch 5/150
3800/3800 [==============================] - 54s 14ms/step - loss: 1.8123 - acc: 0.3911 - val_loss: 2.3226 - val_acc: 0.2600

Epoch 00005: val_loss improve

Epoch 40/150
3800/3800 [==============================] - 54s 14ms/step - loss: 0.6104 - acc: 0.7847 - val_loss: 0.4725 - val_acc: 0.8474

Epoch 00040: val_loss did not improve from 0.46302
Epoch 41/150
3800/3800 [==============================] - 56s 15ms/step - loss: 0.6014 - acc: 0.7858 - val_loss: 0.4638 - val_acc: 0.8432

Epoch 00041: val_loss did not improve from 0.46302
Epoch 42/150
3800/3800 [==============================] - 54s 14ms/step - loss: 0.6342 - acc: 0.7729 - val_loss: 0.4635 - val_acc: 0.8442

Epoch 00042: val_loss did not improve from 0.46302
Epoch 43/150
3800/3800 [==============================] - 54s 14ms/step - loss: 0.6281 - acc: 0.7779 - val_loss: 0.4579 - val_acc: 0.8484

Epoch 00043: val_loss improved from 0.46302 to 0.45795, saving model to model_img.h5
Epoch 44/150
3800/3800 [==============================] - 55s 15ms/step - loss: 0.6301 - acc: 0.7705 - val_loss: 0.4621 - val_acc: 0.8453

Epoch 00044: val_loss did not improve from 0.45795
Epoch 45/150
380

In [15]:
#test데이터 불러오기
n = glob.glob('C:/Users/tnqls/Desktop/plant/test/*.png')
test_img = []
names = []
for a in n:
    if a[-3:] != 'png':
        continue
    names.append(a.split('\\')[-1])
    new_img = Image.open(a)
    test_img.append(ImageOps.fit(new_img, (48, 48), Image.ANTIALIAS).convert('RGB'))

In [18]:
model = load_model('model_batch.h5')

In [19]:
timgs = np.array([np.array(im) for im in test_img])
testX = timgs.reshape(timgs.shape[0], 48, 48, 3) / 255

In [22]:
df = pd.DataFrame(data={'file': names, 'species': test_y})
df_sort = df.sort_values(by=['file'])
df_sort.to_csv('sample_submission.csv', index=False)